In [1]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [2]:
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize
import matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter
import os

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

%matplotlib inline
import os

In [4]:
# Remplace ces imports par les transformations spécifiques à ton problème
# Assure-toi d'avoir les transformations appropriées pour le redimensionnement, la normalisation, etc.
# Vérifie également le nombre de classes dans ton ensemble de données

from torchvision.datasets import ImageFolder

# Définis le chemin vers le répertoire racine de ton ensemble de données extrait
dataset_path = 'flower_images/'

# Définis les transformations à appliquer aux images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ajuste la taille en fonction de tes besoins
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Ajuste la normalisation
])

# Crée l'ensemble de données en utilisant ImageFolder
dataset = ImageFolder(root=dataset_path, transform=transform)

In [5]:
# Divise l'ensemble de données en ensembles d'entraînement et de test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [6]:
# Crée les dataloaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
# Crée les dataloaders
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [34]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2,
            ),  
            nn.ReLU(),
            nn.AdaptiveMaxPool2d(2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),
            nn.ReLU(),
            nn.AdaptiveMaxPool2d(2),
        )
        # Fully connected layer, output 10 classes
        self.out = nn.Linear(32, 100352)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)

        # Take the average over spatial dimensions
        x = x.mean(dim=[2, 3])

        output = self.out(x)
        return output, x

In [35]:
model=CNN()

In [36]:
loss_func = nn.CrossEntropyLoss()   

In [37]:
from torch import optim
optimizer = optim.Adam(model.parameters(), lr = 0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [38]:
def transfo_dataloader_to_tensor(dataloader):
  # Initialiser une liste pour stocker les tensors
  all_data = []
  all_label = []

  # Itérer sur le dataloader et concaténer les mini-lots
  for images, labels in dataloader:
      all_data.append(images)
      all_label.append(labels)

  # Concaténer tous les tensors dans la liste pour obtenir un seul tensor
  all_data_tensor = torch.cat(all_data, dim=0)
  all_label_tensor = torch.cat(all_label, dim=0)
  return all_data_tensor,all_label_tensor

In [13]:
def select_n_random(data, labels, n=100):
    assert len(data) == len(labels)
    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

images, labels = select_n_random(transfo_dataloader_to_tensor(train_dataloader)[0], transfo_dataloader_to_tensor(train_dataloader)[1])
class_labels = [str(i) for i in range(10)]
print(class_labels)
features = images.view(-1, 28 * 28)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [47]:
# Définir la fonction calculate_accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [48]:
from torch.autograd import Variable
num_epochs = 10
run_loss = 0

def train(num_epochs, cnn, train_dataloader, test_dataloader):
  global run_loss  # Déclarer la variable globale run_loss

  total_step = len(train_dataloader)

  # Boucle d'entraînement
  for epoch in range(num_epochs):
    cnn.train()  # Mettez le modèle en mode d'entraînement
    for i, (images, labels) in enumerate(train_dataloader):
      # gives batch data, normalize x when iterate train_loader
      b_x = Variable(images)   # batch x
      b_y = Variable(labels)   # batch y

      output = cnn(b_x)[0]
      loss = loss_func(output, b_y)

      # clear gradients for this training step
      optimizer.zero_grad()

      # backpropagation, compute gradients
      loss.backward()
      run_loss += loss.item()

      # apply gradients
      optimizer.step()

      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, accuracy: {}'
              .format(epoch + 1, num_epochs, i + 1, total_step, run_loss / 100,calculate_accuracy(output, b_y)))
      run_loss = 0.0  # Réinitialiser run_loss après chaque batch

# Assurez-vous d'appeler la fonction train avec les arguments nécessaires
train(num_epochs, model, train_dataloader, test_dataloader)

Epoch [1/10], Step [1/63], Loss: 0.0095, accuracy: 0.65625
Epoch [1/10], Step [2/63], Loss: 0.0096, accuracy: 0.6875
Epoch [1/10], Step [3/63], Loss: 0.0104, accuracy: 0.578125
Epoch [1/10], Step [4/63], Loss: 0.0108, accuracy: 0.578125
Epoch [1/10], Step [5/63], Loss: 0.0078, accuracy: 0.703125
Epoch [1/10], Step [6/63], Loss: 0.0098, accuracy: 0.5625
Epoch [1/10], Step [7/63], Loss: 0.0083, accuracy: 0.71875
Epoch [1/10], Step [8/63], Loss: 0.0108, accuracy: 0.515625
Epoch [1/10], Step [9/63], Loss: 0.0086, accuracy: 0.703125
Epoch [1/10], Step [10/63], Loss: 0.0117, accuracy: 0.484375
Epoch [1/10], Step [11/63], Loss: 0.0100, accuracy: 0.59375
Epoch [1/10], Step [12/63], Loss: 0.0085, accuracy: 0.6875
Epoch [1/10], Step [13/63], Loss: 0.0082, accuracy: 0.65625
Epoch [1/10], Step [14/63], Loss: 0.0080, accuracy: 0.75
Epoch [1/10], Step [15/63], Loss: 0.0109, accuracy: 0.6875
Epoch [1/10], Step [16/63], Loss: 0.0082, accuracy: 0.640625
Epoch [1/10], Step [17/63], Loss: 0.0089, accurac